In [13]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
try:
    import simpleaudio as sa
except:
    %pip install simpleaudio
    import simpleaudio as sa
from IPython.display import Audio
import ipywidgets as widgets
from IPython.display import display
from mpl_toolkits.mplot3d import Axes3D
#import yagmail

In [ ]:
# calculate note frequencies
A_freq = 440
Csh_freq = A_freq * 2 ** (4 / 12)
E_freq = A_freq * 2 ** (7 / 12)

c = 255/np.log(255)

# get timesteps for each sample, T is note duration in seconds
sample_rate = 44100
T = 1
interval = int(sample_rate*T)
t = np.linspace(0, T, interval, False)

# generate sine wave notes
'''def generate_sound(rgb):
    sin_scal = t * 2 * np.pi
    amp_scal = 1/np.log(rgb[0] + rgb[1] + rgb[2])

    def generate_sins(R,G,B):
        R_1 = R*np.sin(A_freq   * sin_scal)
        G_1 = G*np.sin(Csh_freq * sin_scal)
        B_1 = B*np.sin(E_freq   * sin_scal)
        rv = R_1 + G_1 + B_1
        rv*= amp_scal
        return rv

    LOC_WHITE = generate_sins(255,255,255)

    out = generate_sins(rgb[0], rgb[1], rgb[2])
    
    # normalize to 16-bit range
    out*= 32767 / np.max(np.abs(LOC_WHITE))
    return out'''

def generate_sound(rgb):
    sin_scal = t * 2 * np.pi

    def generate_sins(rgb):
        rgb_scal = rgb/255

        max = np.max(rgb_scal)

        R = rgb_scal[0]
        G = rgb_scal[1]
        B = rgb_scal[2]

        R_1 = (50**R / 50**max)*np.sin(A_freq   * sin_scal)
        G_1 = (50**G / 50**max)*np.sin(Csh_freq * sin_scal)
        B_1 = (50**B / 50**max)*np.sin(E_freq   * sin_scal)
        return R_1 + G_1 + B_1
    
    LOC_WHITE = generate_sins(np.array(np.array([255,255,255])))

    out = np.hstack([generate_sins(rgb), np.max(LOC_WHITE)])
    
    # normalize to 16-bit range
    out*= 32767 #/ np.max(np.abs(LOC_WHITE))
    return out



In [ ]:
def generate_random_color():
    return np.random.randint(256, size=3)

def plot_colored_square(rgb):
    col = colors.to_hex(rgb/255)
    x,y = [-1,1,1,-1],[-1,-1,1,1]
    plt.figure(figsize=(3,3))
    plt.axis('equal')
    plt.fill(x, y, col)
    plt.show()

def generate_samples(color_list):
    for c in color_list:
        plot_colored_square(c)
        display(Audio(data = generate_sound(c), rate = sample_rate))
    
color_list = [np.array([255,0,0]), np.array([0,255,0]), np.array([0,0,255]),
              np.array([200,200,0]), np.array([0,200,200]), np.array([200,0,200])]

i = 0
while i < 2:
    c = generate_random_color()
    if np.min(np.linalg.norm(c - color_list, axis = 1)) < 130:
        continue
    else:
        color_list.append(c)
        i += 1

generate_samples(color_list)

In [ ]:
def generate_n_pickers(n):
    #returns np array of N color pickers
    rv = np.array([])
    for i in range(n):
        cp = widgets.ColorPicker(
        concise=True,
        description='Pick a color',
        value='white',
        disabled=False
        )
        rv = np.append(rv, cp)
    return rv

def generate_n_buttons(n):
    #returns np array of N confirmation buttons
    rv = np.array([])
    for i in range(n):
        b = widgets.Button(
        value=False,
        description='Confirm',
        disabled=False,
        tooltip=str(i),
        icon='check'
        )
        rv = np.append(rv, b)
    return rv

def generate_n_colors(n):
    #returns np array of N random colors
    rv = np.array([generate_random_color()])
    for i in range(1,n):
        rv = np.append(rv, [generate_random_color()], axis = 0)
    return rv

def generate_n_sounds(n):
    #returns np array of N sounds from N randomly generated colors, as well as this np array of colors
    rv = np.array([])
    col = generate_n_colors(n)
    for c in col:
        a = Audio(data = generate_sound(c), rate = sample_rate)
        rv = np.append(rv, a)
    return rv, col

N = 10
pickers = generate_n_pickers(N)
buttons = generate_n_buttons(N)
sounds, targets = generate_n_sounds(N)

for i in range(N):
    display(sounds[i], pickers[i], buttons[i])

def on_button_clicked(b):
    #create individual on_button_clicked functions for each button, tying it to corresponding cp
    i = int(b.tooltip)
    pickers[i].disabled = True
    b.disabled = True

for b in buttons:
    b.on_click(on_button_clicked)



In [ ]:
guesses = np.array([np.array(colors.to_rgb(pickers[0].value)) * 255])
for i in range(1,N):
    c = np.array(colors.to_rgb(pickers[i].value)) * 255
    guesses = np.append(guesses, [c], axis = 0)

differences = guesses - targets
distances = np.linalg.norm(differences, axis = 1)
distances

In [ ]:
#%matplotlib widget

fig = plt.figure(figsize=(5,5))
ax = Axes3D(fig)

ax.set_xlim(0,255)
ax.set_ylim(0,255)
ax.set_zlim(0,255)

#Plot red, green, and blue axes as vectors.
ax.quiver(0,0,0,255,0,0, color = 'r')
ax.quiver(0,0,0,0,255,0, color = 'g')
ax.quiver(0,0,0,0,0,255, color = 'b')

for i in range(N):
    g = guesses[i]
    t = targets[i]
    cg = colors.to_hex(g/255)
    ct = colors.to_hex(t/255)
    ax.scatter(g[0], g[1], g[2], c = cg, edgecolors = 'black', s=75, linewidth = 1)
    ax.scatter(t[0], t[1], t[2], c = ct, edgecolors = 'red', s=75, linewidth = 1)
    ax.quiver(g[0], g[1], g[2], t[0] - g[0], t[1] - g[1], t[2] - g[2])


In [15]:
'''user = yagmail.SMTP(user='music128sp22@gmail.com', password='musicproject')
user.send(to='music128sp22@gmail.com', subject='Free iPhone 14 Pro 512 GB', contents='hello')'''

{}

In [16]:
'''import smtplib
import mimetypes
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.message import Message
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText'''

In [17]:
'''emailfrom = "music128sp22@gmail.com"
emailto = "music128sp22@gmail.com"
fileToSend = "GDPC1.csv"
username = "music128sp22@gmail.com"
password = "musicproject"

msg = MIMEMultipart()
msg["From"] = emailfrom
msg["To"] = emailto
msg["Subject"] = "help I cannot send an attachment to save my life"
msg.preamble = "help I cannot send an attachment to save my life"

ctype, encoding = mimetypes.guess_type(fileToSend)
if ctype is None or encoding is not None:
    ctype = "application/octet-stream"

maintype, subtype = ctype.split("/", 1)

if maintype == "text":
    fp = open(fileToSend)
    # Note: we should handle calculating the charset
    attachment = MIMEText(fp.read(), _subtype=subtype)
    fp.close()
elif maintype == "image":
    fp = open(fileToSend, "rb")
    attachment = MIMEImage(fp.read(), _subtype=subtype)
    fp.close()
elif maintype == "audio":
    fp = open(fileToSend, "rb")
    attachment = MIMEAudio(fp.read(), _subtype=subtype)
    fp.close()
else:
    fp = open(fileToSend, "rb")
    attachment = MIMEBase(maintype, subtype)
    attachment.set_payload(fp.read())
    fp.close()
    encoders.encode_base64(attachment)
attachment.add_header("Content-Disposition", "attachment", filename=fileToSend)
msg.attach(attachment)

server = smtplib.SMTP("smtp.gmail.com:587")
server.starttls()
server.login(username,password)
server.sendmail(emailfrom, emailto, msg.as_string())
server.quit()'''

(221,
 b'2.0.0 closing connection y17-20020aa78051000000b0050dc76281b2sm9014405pfm.140 - gsmtp')